In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [17]:
! /opt/bin/nvidia-smi
import pandas as pd
import numpy as np
import tensorflow as tf
import copy
import random
import matplotlib
from matplotlib import pyplot as plt
from time import *
import _pickle as pickle
import json
from sklearn import preprocessing
%matplotlib inline


batch_size = 128
conv1d_length = 17280


Fri Jan  3 14:02:42 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    35W / 250W |   8747MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
content = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/dataframe.csv')
use_column = ['c', 'cap', 'h-l/c', 'ma15min', 'ma30min', 'ma1h',
              'ma2h', 'ma5h', 'ma12h', 'ma24h', 'ma48h', 'cap-ma15min', 'cap-ma30min',
              'cap-ma1h', 'cap-ma2h', 'cap-ma5h', 'cap-ma12h',
              'cap-ma24h', 'cap-ma48h',
              'usdt_huobi', 'out_4h_close']
content = content[use_column]
content = content.dropna(axis=0)

In [0]:
assert list(content.columns) == use_column, 'use_column的column的列的排序不对'

In [18]:
X = np.array(content)[:, :-1]
Y = np.array(content)[:, -1]
X = preprocessing.scale(X)
Y = preprocessing.scale(Y)

mean = np.mean(Y)
std = np.std(Y)
print(f'Y mean: {mean},\nY std: {std}')
Y = (Y - mean) / std

Y mean: -2.839397797857826e-18,
Y std: 1.0000000000000002


In [0]:
class CIFAR10Sequence(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, conv1d_length, batch_size, is_train_set=True, validation_split=0.1, seed=1):
        assert len(x_set) == len(y_set), 'y 不需要进行最后的conv1d的卷积长度的去除'
        self.x, self.y = x_set, y_set[conv1d_length - 1:]
        self.conv1d_length = conv1d_length
        self.batch_size = batch_size
        item_list = [i for i in range(len(self.x) - self.conv1d_length + 1)]
        random.seed(seed)
        self.train_item_list = sorted(random.sample(item_list, int(len(item_list) * (1 - validation_split))))
        if is_train_set:
            self.item_list = self.train_item_list
        else:
            train_item_set = set(self.train_item_list)
            self.test_item_list = [i for i in item_list if i not in train_item_set]
            self.item_list = self.test_item_list

    def __len__(self):
        return len(self.item_list) // self.batch_size

    def __getitem__(self, idx):
        batch_x = [self.x[item_number: item_number + self.conv1d_length] for item_number in self.item_list[idx * self.batch_size: (idx + 1) * self.batch_size]]
        batch_y = [self.y[item_number] for item_number in self.item_list[idx * self.batch_size: (idx + 1) * self.batch_size]]

        return np.array(batch_x), np.array(batch_y)

In [0]:
train = CIFAR10Sequence(X, Y, conv1d_length, batch_size, is_train_set=True)
test = CIFAR10Sequence(X, Y, conv1d_length, batch_size, is_train_set=False)

In [0]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu', input_shape=(conv1d_length,20)),
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    tf.keras.layers.Conv1D(filters=128, kernel_size=3, strides=1, activation='relu'),
    tf.keras.layers.MaxPooling1D(padding='same'),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16,activation="relu"),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1)
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.SGD(0.001),
              loss=tf.keras.losses.MSE,
              metrics = ["mean_squared_error"])

In [0]:
epochs = 55
devide = 1
print(f'epochs: {epochs}, \ndevide: {devide}')
H = model.fit_generator(train, steps_per_epoch=len(train) / devide, epochs=epochs, validation_data=test, validation_steps=len(test) / devide)  # len(train)

epochs: 55, 
devide: 1
Epoch 1/55
4101/4101 [==============================] - 1664s 406ms/step - loss: 1.0037 - mean_squared_error: 1.0037 - val_loss: 0.9893 - val_mean_squared_error: 0.9893
Epoch 2/55
4101/4101 [==============================] - 1668s 407ms/step - loss: 1.0043 - mean_squared_error: 1.0043 - val_loss: 0.9888 - val_mean_squared_error: 0.9888
Epoch 3/55
4101/4101 [==============================] - 1680s 410ms/step - loss: 1.0040 - mean_squared_error: 1.0040 - val_loss: 0.9910 - val_mean_squared_error: 0.9910
Epoch 4/55
4101/4101 [==============================] - 1685s 411ms/step - loss: 1.0040 - mean_squared_error: 1.0040 - val_loss: 0.9890 - val_mean_squared_error: 0.9890
Epoch 5/55
4101/4101 [==============================] - 1685s 411ms/step - loss: 1.0042 - mean_squared_error: 1.0042 - val_loss: 0.9882 - val_mean_squared_error: 0.9882
Epoch 6/55
4101/4101 [==============================] - 1668s 407ms/step - loss: 1.0031 - mean_squared_error: 1.0031 - val_loss: 0.9